In [ ]:
import os
import git
import meilisearch
import uuid
from util import workspace
from infra import searchengine


def get_tracked_files(repo_path):
    """Gitの管理対象になっているファイルの一覧を取得"""
    repo = git.Repo(repo_path)
    return [item[0] for item in repo.index.entries]


def read_file_content(file_path):
    """ファイルの内容を読み込む。バイナリファイルの場合はNoneを返す"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except (UnicodeDecodeError, IOError):
        return None


def create_document(file_path, content):
    """Meilisearch用のドキュメントを作成"""
    return {
        'id': str(uuid.uuid4()),
        'filepath': file_path,
        'content': content,
        'ext': os.path.splitext(file_path)[1][1:] or 'no-extension'
    }


def index_repository(repo_path, meilisearch_url, api_key, index_name):
    """リポジトリ内のファイルをMeilisearchにインデックス"""
    # Meilisearchクライアントの初期化
    client = meilisearch.Client(meilisearch_url, api_key)
    index = client.index(index_name)

    # ファイルの取得とインデックス作成
    tracked_files = get_tracked_files(repo_path)
    documents = []

    for file_path in tracked_files:
        content = read_file_content(os.path.join(repo_path, file_path))
        if content is not None:  # テキストファイルの場合のみインデックス
            doc = create_document(file_path, content)
            documents.append(doc)

    # ドキュメントの追加
    if documents:
        index.add_documents(documents)
        print(f"Indexed {len(documents)} files successfully")
    else:
        print("No text files found to index")
    return client


client = index_repository(workspace.root_dir, 'http://meilisearch:7700',
                          searchengine.MEILI_MASTER_KEY, 'fssearch-mictlan')

Indexed 529 files successfully


In [2]:
client.delete_index('fssearch-mictlan')

TaskInfo(task_uid=395, index_uid='fssearch-mictlan', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2024, 11, 11, 11, 12, 28, 304934))